In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [ ]:
pip install -U langsmith

In [ ]:
pip install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 1.1 MB/s eta 0:00:00


In [ ]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Project Name"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "xxxxxx"
os.environ['OPENAI_API_KEY'] = str("xxxxxx")

In [ ]:
pip install openai

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=1, max_tokens=512, model="text-davinci-003")

In [ ]:
sudoku_puzzle =   "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"
problem_description = f"""
{sudoku_puzzle}

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.
""".strip()
print(problem_description)

3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.


In [ ]:
from typing import Tuple
from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity
import re

class MyChecker(ToTChecker):
    def evaluate(self, problem_description: str, thoughts: Tuple[str, ...] = ()) -> ThoughtValidity:
        last_thought = thoughts[-1]
        clean_solution = last_thought.replace(" ", "").replace('"', "")
        regex_solution = clean_solution.replace("*", ".").replace("|", "\\|")
        if sudoku_solution in clean_solution:
            return ThoughtValidity.VALID_FINAL
        elif re.search(regex_solution, sudoku_solution):
            return ThoughtValidity.VALID_INTERMEDIATE
        else:
            return ThoughtValidity.INVALID

In [ ]:
checker = MyChecker()
assert checker.evaluate("", ("3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1",)) == ThoughtValidity.VALID_FINAL
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1",)) == ThoughtValidity.INVALID

In [ ]:
from langchain_experimental.tot.base import ToTChain

tot_chain = ToTChain(llm=llm, checker=MyChecker(), k=30, c=5, verbose=True, verbose_llm=False)
tot_chain.run(problem_description=problem_description)



> Entering new ToTChain chain...
Starting the ToT solve procedure.
Thought: 3,2,1,4|1,3,4,2|4,1,2,3|4,3,2,1
Thought: 3,2,3,2|1,2,4,3|4,1,2,3|4,3,1,2
Thought: 3,2,4,1|1,3,4,2|4,1,2,3|4,3,2,1
Thought: 3,2,1,4|1,4,3,2|4,1,3,2|4,3,2,1
Thought: 3,2,3,2|1,2,3,4|4,1,2,3|4,3,1,2
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,*,*,1
Thought: 3,*,*,2|1,2,3,*|*,1,*,3|4,*,*,1
    Thought: 3,1,*,2|1,2,3,4|*,1,*,3|4,*,*,1
    Thought: 3,*,1,2|1,2,3,4|*,1,*,3|4,*,*,1
        Thought: 3,2,1,2|1,2,3,4|*,1,*,3|4,*,*,1
        Thought: 3,1,4,2|1,2,3,4|*,1,*,3|4,*,*,1
        Thought: 3,1,3,2|1,2,3,4|*,1,*,3|4,*,*,1
        Thought: 3,1,2,4|1,2,3,4|*,1,*,3|4,*,*,1
        Thought: 3,1,2,3|1,2,4,4|*,1,*,3|4,*,*,1
    Thought: 3,*,*,2|1,2,3,*|4,1,*,3|4,*,*,1
    Thought: 3,*,*,2|1,2,3,*|*,1,4,3|4,*,*,1
        Thought: 3,*,*,2|1,2,3,3|*,1,4,3|4,*,*,1
        Thought: 3,*,*,2|1,2,3,4|*,1,4,3|4,*,*,1
            Thought: 3,1,*,2|1,2,3,4|*,1,4,3|4,*,*,1
            Thought: 3,2,*,

'3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1'